In [ ]:
#vibo: Виталий_Бочаров_stepik_id_56089805, telegram(@VitaliyBocharov)

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
#vibo: загружаем предобученные векторные представления
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2022-03-25 22:25:25--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  12.6MB/s    in 2m 58s  

2022-03-25 22:28:25 (7.78 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



# !!!wv_embeddings

In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
#vibo: каждое слово - вектор размера 200
word = 'dog'
if word in wv_embeddings:
    print('dog', type(wv_embeddings[word]), wv_embeddings[word].dtype, wv_embeddings[word].shape)

dog <class 'numpy.ndarray'> float32 (200,)


In [4]:
#vibo: словарь на 1,8 млн. слов
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [ ]:
# method most_simmilar
'''your code'''

'your code'

In [ ]:
#vbio: векторизованное слово 'dog' ;))))
wv_embeddings['dog']

In [ ]:
#vbio: а Dog нет в словаре, значит нужно делать lower()
wv_embeddings['Dog']

In [7]:
#vibo: импортируемые библиотеки
import numpy as np
import scipy.spatial.distance as ds

#vibo: ФУНКЦИЯ КОСИНУСНОГО РАССТОЯНИЯ (чем меньше, тем лучше)
def cosine_dis(v2):
    return ds.cosine(wv_embeddings['dog'], v2)

#vibo: проверяем косинусное расстояние для одного и того же вектора
print('dog: ', cosine_dis(wv_embeddings['dog']))

#vibo: проверим косинусное расстояние до кошки
print('cat: ', cosine_dis(wv_embeddings['cat']))

dog:  0.0
cat:  0.31476593017578125


In [8]:
#vibo: сделаем тестовый список из 10 первых слов
test_list = wv_embeddings.index2word[0:10]
test_list

['use', 'code', 'using', 'like', 'will', 'want', 'need', 'get', 'file', 'one']

In [ ]:
# #vibo: создадим словарь {слово: косинусное расстояние}
# test_dict = {}

# for i in range(len(test_list)):
#   print(test_list[i])
#   key = test_list[i]
#   print(cosine_dis(wv_embeddings[test_list[i]]))
#   value = cosine_dis(wv_embeddings[test_list[i]])
#   test_dict[key] = value

use
0.9241460710763931
code
0.7885693460702896
using
0.9976603414397687
like
0.8272953629493713
will
0.755998283624649
want
0.7662387192249298
need
0.8126461505889893
get
0.8380052149295807
file
0.8688021153211594
one
0.692000150680542


In [ ]:
# #vibo: создадим словарь {косинусное расстояние: слово}
# test_dict = {}

# for i in range(len(test_list)):
#   #test_dict[test_list[i]] = cosine_dis(wv_embeddings[test_list[i]])
#   test_dict[cosine_dis(wv_embeddings[test_list[i]])] = test_list[i] 

# test_dict

{0.692000150680542: 'one',
 0.755998283624649: 'will',
 0.7662387192249298: 'want',
 0.7885693460702896: 'code',
 0.8126461505889893: 'need',
 0.8272953629493713: 'like',
 0.8380052149295807: 'get',
 0.8688021153211594: 'file',
 0.9241460710763931: 'use',
 0.9976603414397687: 'using'}

In [ ]:
# #vibo: из ключей словаря сделал список, отсортировал по возрастанию
# #sorted(test_dict.values())[0:11]
# sorted_list_keys = sorted(test_dict.keys())[0:6]
# sorted_list_keys

[0.692000150680542,
 0.755998283624649,
 0.7662387192249298,
 0.7885693460702896,
 0.8126461505889893,
 0.8272953629493713]

In [ ]:
# #vibo: выводим значения по ключам
# for i in range(6):
#   print(test_dict[sorted_list_keys[i]])

one
will
want
code
need
like


In [9]:
#vibo: пробую для полного словаря из 1787145 слов...

#vibo: сделаем список
list_1787145 = wv_embeddings.index2word

#vibo: создадим пустой словарь
dict_1787145 = {}

#vibo: на длинну полного списка
for i in range(len(list_1787145)):
  #vibo:  словарь вида {косинусное расстояние: слово}
  dict_1787145[cosine_dis(wv_embeddings[list_1787145[i]])] = list_1787145[i]

#vibo: отсортируем список ключей (косинусное расстояние) по возрастанию, возьмем первые 30
sorted_list_keys = sorted(dict_1787145.keys())[0:31]

#vibo: "нулевое" слово dog и топ-30 ближайших
for i in range(31):
  print('{}: {} ({})'.format(i, dict_1787145[sorted_list_keys[i]], sorted_list_keys[i]))

0: dog (0.0)
1: animal (0.14358210563659668)
2: dogs (0.21191346645355225)
3: mammal (0.2376195788383484)
4: cats (0.23787474632263184)
5: animals (0.23920613527297974)
6: feline (0.26076024770736694)
7: bird (0.26845115423202515)
8: animal1 (0.27807849645614624)
9: doggy (0.2786650061607361)
10: labrador (0.2790868878364563)
11: canine (0.2790943384170532)
12: meow (0.28147053718566895)
13: cow (0.2919555902481079)
14: dog2 (0.2942090630531311)
15: woof (0.294938862323761)
16: dog1 (0.2961159348487854)
17: dog3 (0.29811811447143555)
18: penguin (0.302970826625824)
19: bulldog (0.30595123767852783)
20: mammals (0.3068612217903137)
21: bark (0.3086201548576355)
22: fruit (0.31077486276626587)
23: reptile (0.31087905168533325)
24: furry (0.31365013122558594)
25: carnivore (0.3137050271034241)
26: cat (0.31476593017578125)
27: horse (0.31666189432144165)
28: kitten (0.3179847002029419)
29: sheep (0.3197430372238159)
30: chihuahua (0.3208243250846863)


In [ ]:
#vibo: смущает наличие dog2, dog1, dog3

In [10]:
#vibo: проверим все еще раз по исходной формуле
print('0: dog ({})'.format(cosine_dis(wv_embeddings['dog'])))
print('')
print('Top-5')
print('1: animal ({})'.format(cosine_dis(wv_embeddings['animal'])))
print('2: dogs ({})'.format(cosine_dis(wv_embeddings['dogs'])))
print('3: mammal ({})'.format(cosine_dis(wv_embeddings['mammal'])))
print('4: cats ({})'.format(cosine_dis(wv_embeddings['cats'])))
print('5: animals ({})'.format(cosine_dis(wv_embeddings['animals'])))
print('')
print('Позиция слова "cat"')
#vibo: проверим косинусное расстояние до кошки
print('26: cat ({})'.format(cosine_dis(wv_embeddings['cat'])))

0: dog (0.0)

Top-5
1: animal (0.14358210563659668)
2: dogs (0.21191346645355225)
3: mammal (0.2376195788383484)
4: cats (0.23787474632263184)
5: animals (0.23920613527297974)

Позиция слова "cat"
26: cat (0.31476593017578125)


#### **Ответ на вопрос 1:** итого в top-5 слово 'cat' - не входит, оно находится на 26 месте не считая нулевой позиции 'dog'

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

# !!!base_tokenizer

In [11]:
#vibo: простой токенайзер
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)

base_tokenizer = MyTokenizer()

In [ ]:
# #vibo: посмотрим, что делает регулярное выражение:
# test_str = 'I love neural networks'

# def tokenize(text):
#     return re.findall('\w+', text)

# tokenize(test_str)

['I', 'love', 'neural', 'networks']

In [ ]:
#vibo: вызываю токенайзер
# tokenizer.tokenize('I love neural networks')

['I', 'love', 'neural', 'networks']

In [ ]:
# #vibo: размерность вектора
# (wv_embeddings['networks'].shape)

(200,)

In [ ]:
# #vibo: длина вектора
# len(wv_embeddings['networks'])

200

In [ ]:
# #vibo: нулевой вектор размера 200
# token_vector = np.zeros(200)

In [ ]:
# #vibo: проверяю
# (token_vector).shape

(200,)

# !!!question_to_vec

In [12]:
#vibo: tokenizer ввели выше как tokenizer = MyTokenizer()
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    '''your code'''

    #vibo: используя простой токенайзер, написанный выше токенизируем предложение 
    token_list = tokenizer.tokenize(question)
    #vibo: создаем нулевой вектор длинной dim=200
    token_vector = np.zeros(dim)
    #vibo: идем по списку токенов предложения
    a = 0
    for i in range(len(token_list)):
      #vibo: спомощью контрукции try - except обходим ошибку отсутствия в словаре слова 'I'
      try:
        #token_vector += wv_embeddings[token_list[i]]
        #vibo: заменил, функция не работала без wv_embeddings, выдавая одни нулевые векторы
        token_vector += embeddings[token_list[i]]
        a += 1
      except:
        pass
    #vibo:чтобы не делить на ноль в случае предложения, в котором нет ни одного слова из словаря
    if a > 0:
      return token_vector / a
    elif a == 0:
      return token_vector

In [ ]:
#vibo: заданное по условию предложение
test_question = 'I love neural networks'

#vibo: запускаем написанную функцию для test_question
#vibo: векторное представление слов берем из предобученного представления (вопрос 1)
#vibo: tokenizer - берем из написанного выше класса
#vibo: длина векторов в предобученном словаре 200
question_to_vec(test_question, wv_embeddings, base_tokenizer)

In [ ]:
# #vibo: странно, что проверка без сложения с np.zeros(200) дает false
# #vibo: если кто-то объяснит почему так буду очень благодарен
# question_to_vec(test_question, wv_embeddings, tokenizer) == (wv_embeddings['love'] + wv_embeddings['neural'] + wv_embeddings['networks'])/3

#vibo: странно, что проверка со сложением с np.zeros(200) дает true 
question_to_vec(test_question, wv_embeddings, base_tokenizer) == (np.zeros(200) + wv_embeddings['love'] + wv_embeddings['neural'] + wv_embeddings['networks'])/3
#(np.zeros(200) + wv_embeddings['love'] + wv_embeddings['neural'] + wv_embeddings['networks'])/3

In [ ]:
test_question_error = ' I '

#vibo: по условию задачи если в вопросе нет ни одного слова из словаря, на выходе - нулевой вектор
question_to_vec(test_question_error, wv_embeddings, base_tokenizer)
#vibo: у меня получается деление на ноль, поэтому nan
#vibo: как быстро переделать не понимаю, оставлю пока так

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [16]:
'''your code'''
#vibo: получаем ответ на вопрос 2. с помощью полученной функции
test_question = 'I love neural networks'

round(question_to_vec(test_question, wv_embeddings, base_tokenizer)[2], 2)

-1.29

In [17]:
#vibo: сделаем проверку. 'I' в словаре нет
round(((wv_embeddings['love'])[2] + (wv_embeddings['neural'])[2] + (wv_embeddings['networks'])[2])/3, 2)

-1.29

#### **Ответ на вопрос 2:** -1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

#### **Ответ на вопрос 3:** 1 (если значение метрики Hits находится в интервале [0, 1] и значение метрики DCG в интервале [0, 1], то максимум их разности 1)

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [ ]:
#vibo: расчет по формуле

[K = 10] $\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}}$

In [18]:
import math
#round((1/(math.log2(10))), 1)

import numpy as np
round((1/(np.log2(10))), 1)

0.3

#### **Ответ на вопрос 4**: 0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

# !!!hits_count

In [19]:
#vibo: функция расчета метрики HITS
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    
    hits_value = 0
    #vibo: идем по списку dup_ranks
    for i in dup_ranks:
      #vibo: проверяем условие
      if i <= k:
        hits_value += 1
      else:
        hits_value += 0
    #vibo: делим количество hits_value
    hits_value = hits_value/len(dup_ranks)
    return hits_value    

In [20]:
#vibo: проверим для списка dup_ranks и k 50/50
hits_count([2, 1, 3, 7, 8, 9], 5)

0.5

# !!!dcg_score

In [21]:
import math
#vibo: функция расчета метрики DCG
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''

    dcg_value = 0
    for i in dup_ranks:
      if i <= k:
        dcg_value += 1/(math.log2(1 + i))
      else:
        dcg_value += 0 
    
    dcg_value = dcg_value/len(dup_ranks)
    return dcg_value

In [22]:
# #vibo: проверим для списка dcg_score и k
dcg_score([1, 2, 1, 1, 1, 1], 3)

0.9384882922619097

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [23]:
import pandas as pd

In [24]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
#dup_ranks = ['''your code''']

#vibo: если подразумевается, что candidates_ranking отсортирован от 1 до 4, тогда позиция дубликата вопроса - 2
dup_ranks = [2]
#vibo: может нужно было через написанную выше функцию перевести предложения в векторное представление, а потом найти косинусное расстояние
#vibo: из задачи не ясно.

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [25]:
import pandas as pd
import numpy as np

# correct_answers - метрика для разных k
#vibo: логарифм по основанию 2 можно и из numpy брать, надо запомнить
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [26]:
#vibo: подключаем google-disc в Colab Notebooks
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
#vibo: скачал ахив, выложил на google-disc
#vibo: разархивируем
!unzip '/content/drive/MyDrive/Colab Notebooks/stackoverflow_similar_questions.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [ ]:
# #vibo: делим список на списчок списков по 3 элемента
# l = list(range(100))
# n = 3
# [l[i:i+n] for i in range(0,len(l),n)]

# !!!read_corpus

In [28]:
# #vibo: сделал не правильно - сложил все вопросы в одну большую кучу
# #vibo: а нужно было: <вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>
# #vibo: снова неправильно, надо было делать строку длинной 1000
#vibo: и снова не правильно, нужно было выделить строку длиной 1001

#vibo: нужно было: <вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...(1 + 1000 = 1001)
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        '''your code'''
        #vibo: зашли в строку, которая состоит из нескольких вопросов, разделенных tab
        #vibo: разрезаем строку по tab
        line = line.split('\t')
        #vibo: делаем один большой список
        for question in line:
          #vibo: lower() т.к. в wm_embedings не было заглавных
          data.append(question.lower())
    #vibo: спасибо google:
    #vibo: разбиваем список на список-списков из 1001 элемента (НАДО ЗАПОМНИТЬ)
    n = 1001
    data = [data[i:i+n] for i in range(0,len(data),n)]
    return data

Нам понадобиться только файл validation.

In [29]:
#vibo: подготовливаем данные
validation_data = read_corpus('./data/validation.tsv')

In [ ]:
# #vibo: посмотрим на строку в начале датасета
# validation_data[0]

# #vibo: из конца датасета
# validation_data[-1]

Кол-во строк

In [30]:
#vibo: количество строк в датасете
len(validation_data)

3760

Размер нескольких первых строк

In [31]:
#vibo: проверяем количество элементов в каждой строчке
#vibo: теперь все норм - должно быть 1001, 1 - оригинальный вопрос, 1 - дубликат и 999 - рандомных
#vibo: иначе метрики в предпоследнем задании для больших k не считались
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
#vibo: функция без вычисления косинусного расстояния
# #def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
# def rank_candidates(candidates):
#     """
#         question: строка
#         candidates: массив строк(кандидатов) [a, b, c]
#         result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
#     """
#     '''your code'''

#     #vibo: итоговый список для N=2
#     total_answer = []
#     #vibo: обход списка списков
#     for i in range(len(candidates)):
#       str_answer = []
#       for j in range(len(candidates[i])):
#         #print(j, candidates[i][j])

#         answer = []
#         #vibo: добавляем исходный индекс предложения-кандидата
#         answer.append(j)
#         #vibo: добавляем текст предложения-кандидата
#         answer.append(candidates[i][j])

#         str_answer.append(tuple(answer))
#       #print(str_answer)
#       total_answer.append(str_answer)

#     #vibo: вывод итогового списка    
#     for i in range(len(total_answer)):
#       print('\n{} эксперимент:'.format(i+1))
#       for j in range(len(total_answer[i])):
#         print(total_answer[i][j])

In [ ]:
#vibo: пробую вывести результат без косинусного расстояния
# rank_candidates(candidates)


1 эксперимент:
(0, 'Convert Google results object (pure js) to Python object', 1)
(1, 'C# create cookie from string and send it', 1)
(2, 'How to use jQuery AJAX for an outside domain?', 1)

2 эксперимент:
(0, 'Getting all list items of an unordered list in PHP', 1)
(1, 'WPF- How to update the changes in list item of a list', 1)
(2, 'select2 not displaying search results', 1)


In [ ]:
# #vibo: работает не верно, перемудрил
# from operator import itemgetter

# def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
#     """
#         question: строка
#         candidates: массив строк(кандидатов) [a, b, c]
#         result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
#     """
#     '''your code'''
    
#     #vibo: переводим вопрос в векторное представление
#     test_question = question_to_vec(question, embeddings, tokenizer, dim=200)
#     #print(test_question)

#     #vibo: итоговый список для N=2
#     total_answer = []
#     #vibo: обход списка списков
#     for i in range(len(candidates)):
#       str_answer = []
#       for j in range(len(candidates[i])):
#         #print(j, candidates[i][j])
        
#         #vibo: считаем вектороное представление кандидата
#         test_candidat = question_to_vec(candidates[i][j], embeddings, tokenizer, dim=200)
#         #vibo: считаем косинусное расстояние между 
#         cos_dist = ds.cosine(test_question, test_candidat)

#         answer = []
#         #vibo: добавляем исходный индекс предложения-кандидата
#         answer.append(j)
#         #vibo: добавляем текст предложения-кандидата
#         answer.append(candidates[i][j])
#         #vibo: добавляем косинусное расстояние в список
#         answer.append(cos_dist)
#         #vibo: делаем из списка tuple, добавляем в список для одного эксперимента
#         str_answer.append(tuple(answer))
#       #print(str_answer)
#       #vibo: сортирую список списков по последнему значению вложенного tuple
#       str_answer = sorted(str_answer, key=itemgetter(2))
#       #vibo: добавляю в итоговой список результаты одного экспермента
#       total_answer.append(str_answer)

#     #vibo: вывод итогового списка    
#     for i in range(len(total_answer)):
#       print('\n{} эксперимент:'.format(i+1))
#       for j in range(len(total_answer[i])):
#         print(total_answer[i][j])

In [ ]:
# #vibo: запускаем готовую функцию (исходный индекс, вопрос-кандидат, косунусное расстояние)
# rank_candidates(str(questions), candidates,  wv_embeddings, tokenizer, dim=200)

#vibo: НЕ ВЕРНЫЙ РЕЗУЛЬТАТ! Я СДЕЛАЛ СРАВНЕНИЕ ОДНОГО ВОПРОСА (СКЛЕИЛ ДВА STR) ДЛЯ ДВУХ БЛОКОВ
#vibo: нужно было из списка questions взять один str и сравнивать его с первым блоком вопросов, второй вопрос со вторым
#vibo: за-то разобрался как итерироваться по списку списков и как его сортировать по нужному значению)

In [ ]:
# #vibo: сортировка по списку списков
# from operator import itemgetter
# sorted([(0, 'bla bla 0', 0.4), (1, 'bla bla 2', 0.3), (2, 'bla bla', 0.7)], key=itemgetter(2))

[(1, 'bla bla 2', 0.3), (0, 'bla bla 0', 0.4), (2, 'bla bla', 0.7)]

In [ ]:
# #vibo: проверил тестовый вопрос, норм
# question_to_vec("converting string to list', 'Sending array via Ajax fails", wv_embeddings, tokenizer)

In [ ]:
# #vibo: проверяю косинусное расстояние до 0 предложения эксперимента 1
# ds.cosine(question_to_vec('converting string to list', wv_embeddings, tokenizer),\
#           question_to_vec('Convert Google results object (pure js) to Python object', wv_embeddings, tokenizer))

0.5211371916050211

In [ ]:
# #vibo: проверяю косинусное расстояние до 0 предложения эксперимента 2
# ds.cosine(question_to_vec('Sending array via Ajax fails', wv_embeddings, tokenizer),\
#           question_to_vec('Getting all list items of an unordered list in PHP', wv_embeddings, tokenizer))

0.5430452360795037

In [ ]:
# #vibo: проверяю косинусное расстояние до 1 предложения эксперимента 2
# ds.cosine(question_to_vec('Sending array via Ajax fails', wv_embeddings, tokenizer),\
#           question_to_vec('WPF- How to update the changes in list item of a list', wv_embeddings, tokenizer))

0.5231087593402363

In [ ]:
# #vibo: проверяю косинусное расстояние до 2 предложения эксперимента 2
# ds.cosine(question_to_vec('Sending array via Ajax fails', wv_embeddings, tokenizer),\
#           question_to_vec('select2 not displaying search results', wv_embeddings, tokenizer))

0.8367096496761899

In [ ]:
#vibo: без исключения nan (сортировка по cos_dist в строках с nan не работает)
#vibo: соответсвенно предложение q+ всегда оставалось на 1-позиции, что приводило к завышенным  метрикам

# #vibo: импорт, чтобы ниже произвести сортировку по индексу
# from operator import itemgetter

# def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
#     """
#         question: строка
#         candidates: массив строк(кандидатов) [a, b, c]
#         result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
#     """
#     '''your code'''
    
#     #vibo: переводим вопрос в векторное представление
#     test_question = question_to_vec(question, embeddings, tokenizer, dim=200)
#     #print(test_question)

#     #vibo: обход списка
#     str_answer = []   
#     for i in range(len(candidates)):
#         #vibo: считаем вектороное представление кандидата
#         test_candidat = question_to_vec(candidates[i], embeddings, tokenizer, dim=200)
#         #vibo: считаем косинусное расстояние между 
#         cos_dist = ds.cosine(test_question, test_candidat)
#         answer = []
#         #vibo: добавляем исходный индекс предложения-кандидата
#         answer.append(i)
#         #vibo: добавляем текст предложения-кандидата
#         answer.append(candidates[i])
#         #vibo: добавляем косинусное расстояние в список
#         answer.append(cos_dist)
#         #vibo: добавляем в итоговый список список индекс - вопрос
#         str_answer.append(answer)
#     #vibo: сортирую список по последнему значению (косинусное расстояние)
#     str_answer = sorted(str_answer, key=itemgetter(2))
#     #vibo: теперь из каждого вложенного списка нужно убрать cos_dist
#     # for i in str_answer:
#     #   cos_dist = i.pop(2)

#     #vibo: как я понял по условию задачи делаем (индекс,  предложение) tuple
#     total_list = []
#     for i in str_answer:
#       result = tuple(i)
#       total_list.append(result)

#     return total_list

# !!!rank_candidates

In [33]:
#vibo: импорт, чтобы ниже произвести сортировку по индексу
#vibo: с исключением строк с cos_dist = nan (иначе сортировка не работает)
#vibo: введя фильтр nan для обучаемых эмбедингов получил исключение одной строки q+ и все сломалось

import math
from operator import itemgetter

def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''
    
    #vibo: переводим вопрос в векторное представление
    test_question = question_to_vec(question, embeddings, tokenizer, dim=200)
    #print(test_question)

    #vibo: обход списка
    str_answer = []   
    for i in range(len(candidates)):
        #vibo: считаем вектороное представление кандидата
        test_candidat = question_to_vec(candidates[i], embeddings, tokenizer, dim=200)
        #vibo: считаем косинусное расстояние между 
        cos_dist = ds.cosine(test_question, test_candidat)
        answer = []
        #vibo: добавляем исходный индекс предложения-кандидата
        answer.append(i)
        #vibo: добавляем текст предложения-кандидата
        answer.append(candidates[i])
        #vibo: добавляем косинусное расстояние в список
        answer.append(cos_dist)
        
        # #vibo: проверка на nan
        # if math.isnan(cos_dist):
        #   pass
        # else:
        #   #vibo: добавляем в итоговый список список индекс - вопрос
        #   str_answer.append(answer)

        str_answer.append(answer)
    
    #vibo: сортирую список по последнему значению (косинусное расстояние)
    str_answer = sorted(str_answer, key=itemgetter(2))
    #vibo: теперь из каждого вложенного списка нужно убрать cos_dist
    # for i in str_answer:
    #   cos_dist = i.pop(2)

    #vibo: как я понял по условию задачи делаем (индекс,  предложение) tuple
    total_list = []
    for i in str_answer:
      result = tuple(i)
      total_list.append(result)

    return total_list

In [ ]:
# #vibo: отрезаем элемент tuple (потом переделал на list и убрал через pop)
# str_answer = [(0, 'bla bla 0', 0.4), (1, 'bla bla 2', 0.3), (2, 'bla bla 3', 0.7)]
# for i in range(len(str_answer)):
#       result = str_answer[i][0:2]
#       print(result)

(0, 'bla bla 0')
(1, 'bla bla 2')
(2, 'bla bla 3')


In [ ]:
# # #vibo: отрезаем элемент tuple (потом переделал на list и убрал через pop)
# t = (0, 'bla bla 0', 0.4)
# t = t[:-1]
# t

(0, 'bla bla 0')

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [34]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [35]:
#vibo: проверяем результат для эксперимента 1 - 102 (верно), для эксперимента 2 - 102
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, base_tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it', 0.453709024768017), (0, 'Convert Google results object (pure js) to Python object', 0.5211371916050211), (2, 'How to use jQuery AJAX for an outside domain?', 0.838115468996391)]

[(1, 'WPF- How to update the changes in list item of a list', 0.5231087593402363), (0, 'Getting all list items of an unordered list in PHP', 0.5430452360795037), (2, 'select2 not displaying search results', 0.8367096496761899)]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

#### **Ответ на вопрос 5**: 102

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [36]:
from tqdm.notebook import tqdm

#vibo: проходим по заготовленным данным, ограничиваясь 1000 строк
#vibo: вычисляем для каждого предложения cos_dist, сортируем по возрастанию и определеяем итоговую позицию q+ 
wv_ranking = []
max_validation_examples = 1000
#vibo: формируем данные для подачи в rank_candidates
for i, line in enumerate(tqdm(validation_data)):
    #vibo: вводим ограничение на 1000 строк
    if i == max_validation_examples:
        break
    #vibo: используем tqdm, делаем список q и список example (1000)
    q, *ex = line

    #vibo: вызываем функцию rank_candidates
    #vibo: функция возвращает порядковый номер вопроса q+, с индексом 0
    #vibo: главное было сделать строку из 1001 элемента, 1 вопрос + 1000 example (с одним оригинальным дубликатом)
    ranks = rank_candidates(q, ex, wv_embeddings, base_tokenizer)

    # #wv_ranking.append([r[0] for r in ranks].index('0') + 1)
    # #vibo: спасибо чату https://t.me/dls_spring2022_part2
    #vibo: делаем список из рейтинга предложений
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
# #vibo: смотрим на итоги сортировки последнего эксперимента
# ranks

In [37]:
#vibo: подсчет номеров
from collections import Counter
print(Counter(wv_ranking))
#vibo: итого из 1000 экспериментов модель 415 раз верно нашла истинный дубликат вопроса

Counter({1: 415, 2: 67, 3: 45, 4: 33, 5: 22, 6: 19, 8: 15, 10: 13, 12: 13, 7: 11, 9: 11, 13: 11, 11: 9, 14: 8, 16: 7, 41: 7, 34: 7, 51: 6, 17: 6, 18: 6, 23: 5, 25: 5, 20: 5, 42: 4, 91: 4, 67: 4, 128: 4, 31: 4, 83: 4, 29: 4, 64: 4, 47: 4, 49: 4, 26: 3, 28: 3, 15: 3, 76: 3, 50: 3, 149: 3, 21: 3, 39: 3, 57: 3, 24: 3, 54: 3, 68: 3, 81: 3, 22: 3, 118: 3, 55: 3, 126: 2, 62: 2, 112: 2, 44: 2, 108: 2, 308: 2, 30: 2, 170: 2, 160: 2, 289: 2, 447: 2, 198: 2, 46: 2, 100: 2, 176: 2, 136: 2, 71: 2, 106: 2, 65: 2, 43: 2, 74: 2, 111: 2, 220: 2, 183: 2, 59: 2, 37: 2, 520: 1, 325: 1, 79: 1, 695: 1, 78: 1, 222: 1, 141: 1, 615: 1, 104: 1, 97: 1, 269: 1, 876: 1, 115: 1, 92: 1, 613: 1, 27: 1, 252: 1, 73: 1, 563: 1, 58: 1, 483: 1, 143: 1, 410: 1, 217: 1, 534: 1, 145: 1, 102: 1, 680: 1, 119: 1, 669: 1, 718: 1, 105: 1, 38: 1, 163: 1, 699: 1, 477: 1, 63: 1, 150: 1, 574: 1, 311: 1, 253: 1, 48: 1, 154: 1, 493: 1, 125: 1, 835: 1, 85: 1, 60: 1, 211: 1, 510: 1, 276: 1, 193: 1, 568: 1, 485: 1, 87: 1, 270: 1, 312: 1, 

In [38]:
#vibo: расчет метрик для разных k
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.525 | Hits@  10: 0.651
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000


In [ ]:
#vibo: обе метрики возрастают с ростом k (абсолютный максимум метрик в 1, минимум 0)
#vibo: в данном эксперименте мы проверяем какое место в итоговом списке занял вопрос - оригинальный дубликат
#vibo: всего мест 1000, теперь к метрикам:

#vibo: метрика Hits говорит о том сколько раз оригинальный дубликат попал в интервал [1(первое место), k (k-тое место)]
#vibo: видно, что с увеличением k метрика увеличивается и не доходя до k = 1000, при k=500 становится равной 1
#vibo: это говорит о том, что ниже 500 места наш оригинал-дубликат не занимал

#vibo: чем наш оригинал дубликат ближе к 1 месту, тем метрика ближе к 1

#vibo: после добавления lower() в read_corpus метрики ухудшились, но это более корректный результат

### Эмбеддинги, обученные на корпусе похожих вопросов

In [39]:
#vibo: вызываем read_corpus (выше вызывали для validation)
train_data = read_corpus('./data/train.tsv')

In [40]:
len(train_data)

2255

In [41]:
len(train_data[0])

1001

In [42]:
train_data[0][0:2]

['converting string to list',
 'convert google results object (pure js) to python object\n']

In [43]:
train_data[1][0:2]

['messagequeueexception (0x80004005): access to message queuing system is denied',
 'win service getting permission denied to message queuing\n']

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [44]:
'''your code'''

#vibo: сбственно выше вызвал функцию read_corpus но уже для трейн дата
#vibo: получили строчки по 1001 элементу из которых оставить нужно только первые два 
#vibo: т.е. оригинальный вопрос и дубликат - делаем обучающую выборку, от каждой строки нужно отрезать вопрос q и q+
total_res = []
#vibo: идем по списку и склеиваем первое и второе предложения
for i in train_data:
    # print(i[0])
    # print(i[1])
    res = i[0] + ' ' + i[1]
    total_res.append(res)
    # print(total_res)

In [45]:
#vibo: посмотрим как выглядит обучающий датасет
total_res[0:6]
#vibo: когда мы склеили две строки в одну (q_оригинал и q+_истиный дубликат) - дали возможность модели найти закономерность в схожести этих вух строк

['converting string to list convert google results object (pure js) to python object\n',
 'messagequeueexception (0x80004005): access to message queuing system is denied win service getting permission denied to message queuing\n',
 'how to run my own external command in python script\n could not load file or assembly microsoft.sqlserver.management.sdk.sfc version 11.0.0.0',
 "rails sends 0 byte files using send_file\n what's happening in queryset converting to list",
 'strange android recyclerview (gridlayoutmanager) focus behavior recyclerview focus scrolling\n',
 'php date range array for the last 40 sundays\n objective c - how to know if point is inside a quarter of circle?']

# !!! pymorphy2.MorphAnalyzer (нормализация -> лемматизация)

In [ ]:
# #vibo: делаем лемматизацию - процесс приведения словоформы к лемме — её нормальной (словарной) форме

# #vibo: библиотека pymorphy2 - только отдельное слово, контекст не применяется поэтому строчку сплитим
# #vibo: флаг -q чтобы не было лишних логов
# !pip install -q pymorphy2

# import pymorphy2
# morph = pymorphy2.MorphAnalyzer()

# words = []
# for line in total_res:
#   #print(line)
#   pymorphy_results = list(map(lambda x: morph.parse(x), line.split(' ')))
#   #print(pymorphy_results)
#   new_string =(' '.join([res[0].normal_form for res in pymorphy_results]))
#   #print(new_string)
#   tokens = base_tokenizer.tokenize(new_string)
#   #print(tokens)
#   words.append(tokens)

# #vibo: результат относительно стекинга PorterStemmer ухудшился
# '''
# DCG@   1: 0.152 | Hits@   1: 0.152
# DCG@   5: 0.201 | Hits@   5: 0.246
# DCG@  10: 0.217 | Hits@  10: 0.297
# DCG@ 100: 0.268 | Hits@ 100: 0.558
# DCG@ 500: 0.318 | Hits@ 500: 0.941
# DCG@1000: 0.324 | Hits@1000: 1.000
# '''

# !!! spacy (нормализация -> лемматизация)

In [ ]:
# #vibo: делаем лемматизацию - процесс приведения словоформы к лемме — её нормальной (словарной) форме

# #vibo: пробую применить библиотека spacy работает контекстно, это нейросетевый подход (не работает с русским в открытом доступе)
# #vibo: контекст учитывается, поэтому сплитить строку не нужно

# import spacy
# #vibo: создаем объект nlp
# nlp = spacy.load('en')

# words = []
# for line in total_res:
#   #vibo: закидываем в объект nlp текст
#   spacy_results = nlp(line)
#   #vibo: получаем начальные формы
#   new_string = (' '.join([token.lemma_ for token in spacy_results]))
#   tokens = base_tokenizer.tokenize(new_string)
#   #print(tokens)
#   words.append(tokens)

# #vibo: результат относительно pymorphy2 улучшился, но стемминг был лучше
# '''
# DCG@   1: 0.159 | Hits@   1: 0.159
# DCG@   5: 0.221 | Hits@   5: 0.277
# DCG@  10: 0.237 | Hits@  10: 0.326
# DCG@ 100: 0.302 | Hits@ 100: 0.660
# DCG@ 500: 0.344 | Hits@ 500: 0.979
# DCG@1000: 0.346 | Hits@1000: 1.000
# '''

'\nDCG@   1: 0.159 | Hits@   1: 0.159\nDCG@   5: 0.221 | Hits@   5: 0.277\nDCG@  10: 0.237 | Hits@  10: 0.326\nDCG@ 100: 0.302 | Hits@ 100: 0.660\nDCG@ 500: 0.344 | Hits@ 500: 0.979\nDCG@1000: 0.346 | Hits@1000: 1.000\n'

In [ ]:
# #vibo: работа токенайзера из начала ноутбука для первой строки
# print(base_tokenizer.tokenize('converting string to list Convert Google results object (pure js) to Python object\n'))

['converting', 'string', 'to', 'list', 'Convert', 'Google', 'results', 'object', 'pure', 'js', 'to', 'Python', 'object']


# !!! base_tokenizer vs nltk.tokenize

In [46]:
#vibo: каждую строку нужно токенизировать для подачи в Word2Vec
words = [base_tokenizer.tokenize(line) for line in total_res]

In [21]:
# import nltk
# nltk.download('punkt')
# from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
# words = [word_tokenize(line) for line in total_res]

In [47]:
print(words[1])

['messagequeueexception', '0x80004005', 'access', 'to', 'message', 'queuing', 'system', 'is', 'denied', 'win', 'service', 'getting', 'permission', 'denied', 'to', 'message', 'queuing']


# !!! PorterStemmer vs. SnowballStemmer  (нормализация -> стемминг)

In [48]:
#vibo: стемминг - процесс нахождения основы слова для заданного исходного слова
from nltk.stem import PorterStemmer

#vibo: вариант 1. PorterStemmer привел к улучшению
ps = PorterStemmer()
words = [list(map(ps.stem, line)) for line in words]
# '''
# DCG@   1: 0.189 | Hits@   1: 0.189
# DCG@   5: 0.249 | Hits@   5: 0.302
# DCG@  10: 0.269 | Hits@  10: 0.363
# DCG@ 100: 0.346 | Hits@ 100: 0.760
# DCG@ 500: 0.379 | Hits@ 500: 0.999
# DCG@1000: 0.379 | Hits@1000: 1.000
# '''

# # from nltk.stem import SnowballStemmer

# # #vibo: вариант 2. SnowballStemmer тоже привел к улучшению, но чуть хуже
# # ss = SnowballStemmer(language='english')
# # words = [list(map(ss.stem, line)) for line in words]
# # '''
# # DCG@   1: 0.181 | Hits@   1: 0.181
# # DCG@   5: 0.242 | Hits@   5: 0.297
# # DCG@  10: 0.262 | Hits@  10: 0.356
# # DCG@ 100: 0.343 | Hits@ 100: 0.778
# # DCG@ 500: 0.373 | Hits@ 500: 1.000
# # DCG@1000: 0.373 | Hits@1000: 1.000
# # '''

In [49]:
#vibo: смотрим результат
print(words[1])

['messagequeueexcept', '0x80004005', 'access', 'to', 'messag', 'queu', 'system', 'is', 'deni', 'win', 'servic', 'get', 'permiss', 'deni', 'to', 'messag', 'queu']


In [50]:
len(words)

2255

In [51]:
#vibo: после всей предобработки делаем эмбединги на трейн дате в Word2Vec
from gensim.models import Word2Vec

#vibo: размер эмбединга size, минимальное количество вхождений слова в корпус min_count, окно, которым идем по тексту window
#vibo: window - окно, с помощью которого пытаемся понять контекст в котором используется слово
#vibo: окно размера (n) значит берем по (n-1) слова справа и слева (для n=3: контект_слово, контекст_слово, слово, контекст_слово, контекст_слово)
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5,
                 #vibo: поменял training algorithm с CBOW на skip-gram (по умолчанию CBOW или sg=0), стало хуже
                 sg=0).wv

#vibo: два принципа работы w2vec
#vibo: CBOW (Continuous Bag of Words) - пытаемся предсказать целевое слово с помощью контекста,
#vibo: обычно лучше работает с небольшими датасетами. У нас всего 2255 предложений в обучающей выборке (со skip-gram метрики заметно упали)
#vibo: Skip-Gram — пытаемся предсказать контекстные слова с помощью целевого слова. Skip-gram лучше работает на больших наборах данных.

In [ ]:
# #vibo: при приведении к lower() сортировка работает
# rank_candidates('string', ['python', 'time', 'string'], embeddings_trained, base_tokenizer)

[(2, 'string', 0.0),
 (0, 'python', 5.486424873457629e-05),
 (1, 'time', 7.492115228524554e-05)]

In [ ]:
# #vibo: пример
# import numpy as np
# import scipy.spatial.distance as ds
# v1 = np.array([-1.01422749, -1.68912624, -1.28541217, -1.37103013,  0.15916634])
# v2 = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
# ds.cosine(v1, v2)

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


nan

In [ ]:
# #vibo: с учетом работы rank_candidates нужно в read_corpus обязательно делать lower()
# #vibo: иначе получается вот такая хрень (нулевые строки дают cos_dist = nan, и сортировка rank_candidates не работает
#vibo: а это приводит к завышенным метрикам, т.к. q+ всегда получается на 1-месте
# rank_candidates('line', ['Python', 'Time', 'string', 'Time', 'line'], embeddings_trained, base_tokenizer)
# #vibo: слова Python, Time - нет в словаре, их векторное представление 0 вектров dim=200
# #vibo: косинусное расстояние до такого объекта nan

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


[(0, 'Python', nan),
 (1, 'Time', nan),
 (2, 'string', 6.827845252488363e-05),
 (3, 'Time', nan),
 (4, 'line', 0.0)]

In [52]:
#vibo: ранжируем строки из validation_data с применением обученного embeddings_trained 
from tqdm.notebook import tqdm
import numpy as np
import scipy.spatial.distance as ds

wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, base_tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [53]:
#vibo: embeddings_trained если включаю фильтр nan в rank_candidates на cos_dist то из 1000 остается 997 строк 
#vibo: (т.е. три эксперимента не корректные), при этом одна из этих трех строк наша q+ с base-line код ломается
#vibo: предложение q+ получается нулевым, если в нем не нашлось ни одного слова из словаря.
#vibo: три битых резульата из 1000 не так страшно. на будущее нужно контролировать количество нулевых строк.
len(ranks)

1000

In [54]:
#vibo: подсчет номеров
from collections import Counter
print(Counter(wv_ranking))

Counter({1: 189, 2: 48, 3: 33, 4: 19, 7: 17, 6: 15, 5: 13, 20: 13, 13: 11, 14: 11, 34: 11, 9: 11, 10: 11, 16: 9, 11: 9, 18: 8, 15: 8, 21: 8, 46: 8, 43: 8, 39: 7, 25: 7, 8: 7, 22: 7, 122: 7, 38: 7, 59: 7, 116: 6, 146: 6, 49: 6, 90: 6, 56: 6, 12: 6, 30: 6, 36: 6, 37: 6, 44: 6, 24: 6, 50: 5, 62: 5, 83: 5, 35: 5, 31: 5, 105: 5, 19: 5, 40: 5, 61: 5, 75: 5, 41: 5, 17: 5, 55: 5, 27: 5, 51: 4, 158: 4, 159: 4, 47: 4, 58: 4, 45: 4, 101: 4, 54: 4, 64: 4, 78: 4, 80: 4, 23: 4, 42: 4, 112: 4, 32: 4, 57: 4, 104: 4, 100: 4, 28: 4, 106: 4, 48: 4, 71: 4, 29: 4, 79: 3, 170: 3, 215: 3, 125: 3, 238: 3, 181: 3, 96: 3, 66: 3, 60: 3, 92: 3, 114: 3, 97: 3, 87: 3, 73: 3, 210: 3, 76: 3, 127: 3, 26: 3, 199: 3, 33: 3, 53: 3, 81: 3, 124: 3, 68: 3, 113: 3, 119: 3, 162: 3, 118: 3, 197: 3, 163: 3, 95: 2, 175: 2, 70: 2, 168: 2, 165: 2, 85: 2, 63: 2, 154: 2, 283: 2, 130: 2, 242: 2, 94: 2, 143: 2, 82: 2, 173: 2, 103: 2, 164: 2, 156: 2, 99: 2, 227: 2, 117: 2, 218: 2, 187: 2, 147: 2, 109: 2, 179: 2, 174: 2, 128: 2, 133: 2,

In [55]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.189 | Hits@   1: 0.189
DCG@   5: 0.249 | Hits@   5: 0.302
DCG@  10: 0.269 | Hits@  10: 0.363
DCG@ 100: 0.346 | Hits@ 100: 0.760
DCG@ 500: 0.379 | Hits@ 500: 0.999
DCG@1000: 0.379 | Hits@1000: 1.000


In [ ]:
#vibo: итого. результат сильно хуже, чем с предобученными эмбедингами.
#vibo: при фиксированном min_count=5 с ростом window метрики улучшаются, при window=5 max, потом ухудшаются
#vibo: прификсированном window=5, с при уменьшении min_count до 3 метрики ухудшаются,
#vibo:при увеличении до min_count=6 незначительно увеличиваются
#vibo: на данный момент оптимальные параметры min_count=5, window=5

#vibo: пробовал другой токенайзер (WordPunctTokenizer) не помог, метрики ухудшились
# '''
# Параметры Word2Vec:
#                       min_count=5
#                       window=5
# Результаты:

# DCG@   1: 0.108 | Hits@   1: 0.108
# DCG@   5: 0.138 | Hits@   5: 0.166
# DCG@  10: 0.152 | Hits@  10: 0.208
# DCG@ 100: 0.204 | Hits@ 100: 0.478
# DCG@ 500: 0.261 | Hits@ 500: 0.918
# DCG@1000: 0.269 | Hits@1000: 1.000
# '''

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


vibo:
* Какой принцип токенизации даёт качество лучше и почему? - **токенизация по словам**;
* Помогает ли нормализация слов? - **да, стемминг улучшил метрики в большей степени, лемминг в меньшей**;
* Какие эмбеддинги лучше справляются с задачей и почему? - **предобученные (т.к. получены на большей обучающей выборке)**;
* Почему получилось плохое качество решения задачи? - **небольшой набор обучающей выборки, нужны более сложные модели**;
* Предложите свой подход к решению  - **увеличить объем обучающей выборки, использовать для обучения полноценную нс**.

In [ ]:
#vibo: Виталий_Бочаров_stepik_id_56089805, telegram(@VitaliyBocharov)
#26.03.2022